In [2]:
import azure.cognitiveservices.speech as speechsdk
import config

speech_key = config.SPEECH_KEY
service_region = config.SERVICE_REGION

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

# Rest of your code
audio_file_path = "sampleaudio.wav"
audio_input = speechsdk.AudioConfig(filename=audio_file_path)
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)
result = speech_recognizer.recognize_once()

if result.reason == speechsdk.ResultReason.RecognizedSpeech:
    print("Transcription:", result.text)
elif result.reason == speechsdk.ResultReason.NoMatch:
    print("No speech could be recognized.")
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Speech Recognition canceled:", cancellation_details.reason)
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("Error details:", cancellation_details.error_details)

Transcription: Proceedings needed to plant the spring corn.


In [4]:
import io
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import config1

subscription_key = config1.AZURE_SUBSCRIPTION_KEY
endpoint = config1.AZURE_ENDPOINT

client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

def analyze_image_azure(image_path):
    with open(image_path, 'rb') as image_file:
        image_data = image_file.read()

    image_stream = io.BytesIO(image_data)

    analysis = client.analyze_image_in_stream(image_stream, visual_features=['Objects'])

    print('Objects detected (Azure Vision):')
    for obj in analysis.objects:
        print(f'{obj.object_property} (confidence: {obj.confidence})')

azure_image_file_path = 'rusty.jpg'
analyze_image_azure(azure_image_file_path)

Objects detected (Azure Vision):
person (confidence: 0.88)
person (confidence: 0.762)


In [3]:
import os
from google.cloud import vision
from google.cloud import storage

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gcp-key.json"

vision_client = vision.ImageAnnotatorClient()

storage_client = storage.Client()

def analyze_image_gcp(image_uri):
    image = vision.Image()
    image.source.image_uri = image_uri

    response = vision_client.object_localization(image=image)
    objects = response.localized_object_annotations

    print('Objects detected (GCP Vision):')
    for object_ in objects:
        print(f'{object_.name} (confidence: {object_.confidence:.2f})')

    if response.error.message:
        raise Exception(f'{response.error.message}')

bucket_name = 'hha-main-bucket'
image_filename = 'rusty.jpg'  
image_uri = f'gs://{bucket_name}/{image_filename}'

analyze_image_gcp(image_uri)